<h1 align = "center">CRUD Operation for FOREX Rates</h1>

In [7]:
import os # miscellaneous os interfaces

In [1]:
import datetime as dt

In [6]:
from tqdm import tqdm as TQ # progress bar for loops

In [2]:
import pandas as pd

%precision 3
pd.set_option('display.max_rows', 50) # max. rows to show
pd.set_option('display.max_columns', 15) # max. cols to show
pd.options.display.float_format = '{:,.3f}'.format # float precisions

In [3]:
import forexrates

In [19]:
import connections # https://gist.github.com/ZenithClown/9700b120ec6cb1acb5ce7bdf4fe65dbf
import datetime_ as dt_ # https://ds-gringotts.readthedocs.io/en/latest/dt_utils.html

## Argument Defination & Control

The FOREX Rates can be fetched from [ExchangeRatesIO](https://exchangeratesapi.io/) and the same can be inserted into the database table. In this notebook, we've considered the database schema to follow the structure of [**`dataway`**](https://github.com/ZenithClown/dataway), else please modify the structure as per your requirement. The transaction table considers the schema (MS-SQL Server) like:

```sql
CREATE TABLE [meta].[currency_code] (
    [currency_code] CHAR(3),
    [currency_name] VARCHAR(64) NOT NULL,

    -- ISO 4217 - Numeric Code for a Currency is also Unique
    [numeric_code] CHAR(3) NOT NULL,
    [decimal_unit] INTEGER DEFAULT 0,

    CONSTRAINT [pk_currency_code] PRIMARY KEY NONCLUSTERED ([currency_code]),

    CONSTRAINT [uq_currency_name] UNIQUE ([currency_name]),
    CONSTRAINT [uq_currency_numeric_code] UNIQUE ([numeric_code])
);


CREATE TABLE [dbo].[foreign_exchange_rate] (
    [_id] INTEGER PRIMARY KEY IDENTITY(1, 1),

    -- currency is stored in the format <date>, <base>, <code>, <rate>
    -- in addition, the data source is tracked to validate between different source
    [effective_date]        DATE NOT NULL,
    [base_currency_code]    CHAR(3) NOT NULL,
    [target_currency_code]  CHAR(3) NOT NULL,
    [foreign_exchange_rate] NUMERIC(18, 6) NOT NULL,

    [data_source_id] CHAR(5) NOT NULL,

    -- ? Sanity Check - Only one record for a date/currency pair from a data source
    CONSTRAINT [uq_forex_record_source] UNIQUE(
        [effective_date]
        , [data_source_id]
        , [base_currency_code]
        , [target_currency_code]
    ),

    CONSTRAINT [fk_base_currency_code] FOREIGN KEY ([base_currency_code])
        REFERENCES [meta].[currency_code] ([currency_code])
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,

    CONSTRAINT [fk_target_currency_code] FOREIGN KEY ([target_currency_code])
        REFERENCES [meta].[currency_code] ([currency_code])
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,

    CONSTRAINT [fk_forex_data_source_id] FOREIGN KEY ([data_source_id])
        REFERENCES [meta].[data_source] ([source_id])
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
);

```

In [5]:
start, end = dt.date(2025, 1, 1), dt.date(2025, 1, 12)
dates = list(dt_.date_range(start, end)) # please refer gist documentation

In [16]:
EXCHANGE_RATES_API_KEY = os.getenv("EXCHANGE_RATES_API_KEY") or input("Enter API Key for Exchange Rates IO (exchangeratesapi.io): ")

In [18]:
rates = pd.concat([
    forexrates.api.ExchangeRatesAPI(apikey = EXCHANGE_RATES_API_KEY, endpoint = date.strftime("%Y-%m-%d")) \
        .get(parsewith = forexrates.io.dataframe.exchangeratesio) for date in TQ(dates, desc = f"Fetching FOREX Rates from {start} to {end} (= {(end - start).days}):")
], ignore_index = True)

rates["data_source_id"] = "ERAPI"
rates.sample(3)

Fetching FOREX Rates from 2025-01-01 to 2025-01-12 (= 11):: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:05<00:00,  2.11it/s]


,effective_date,base_currency_code,target_currency_code,foreign_exchange_rate,data_source_id
376,2025-01-03,EUR,CUC,1.031,ERAPI
1565,2025-01-10,EUR,CRC,517.624,ERAPI
454,2025-01-03,EUR,PEN,3.872,ERAPI


In [20]:
conn, cur, engine, server = connections.connect(...)

In [21]:
rates.to_sql("foreign_exchange_rate", engine, schema = "dbo", index = False, if_exists = "append") # insert into database

-1